# Code Challenge - Una Health

## ⭐ Data

We've prepared sample data for three sample patients: `aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa`, `bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb` and `cccccccc-cccc-cccc-cccc-cccccccccccc`. Each patient is identified by a [universally unique identifier](https://en.wikipedia.org/wiki/Universally_unique_identifier) (UUID).

The data consists of:

- A `csv` file with the historic blood glucose levels of the patient: `levels_all.csv`. We're interested in blood glucose readings which have an `Aufzeichnungstyp` of either `0` or `1` (those appear in different columns because they are different types, automatically collected every 15 minutes vs. manual scanned by the patient, but are readings from the same sensor and thus should be treated as such). The blood glucose reading is noted in `Glukosewert-Verlauf mg/dL` or `Glukose-Scan mg/dL`. All timestamps noted in this file are UTC.
- A `csv` file with the tracked meals of the patient: `activities_all.csv`. Each meal is identified by a UUID.

## 📉 Tasks
- **Visualise** - Create plots for the historic blood glucose level by each patient and the historic blood glucose level after meals (we usually look at `timestamp_start` of the meal + 3 hours worth of data). Feel free to group and slice the data for the individual meals as you see fit.

- **Interpret** - What conclusions can you draw when looking at the combined data of historic blood glucose levels and tracked meals for an individual patient and for certain meal types of an individual patient? What clusters (if any) do you find? What additional information (if any) do you need? What clustering methods would you apply?

In [1]:
#Libraries
import pandas as pd
import numpy as np
import dateutil.parser
from datetime import datetime, timezone, timedelta
import plotly.express as px
import re

In [2]:
#Data loading
data_path = 'data'
a_UUID = 'aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa'
b_UUID = 'bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb'
c_UUID = 'cccccccc-cccc-cccc-cccc-cccccccccccc'
a_activities_all = pd.read_csv(f'{data_path}/{a_UUID}/activities_all.csv', header=0).dropna(axis=1, how='all')
b_activities_all = pd.read_csv(f'{data_path}/{b_UUID}/activities_all.csv', header=0).dropna(axis=1, how='all')
c_activities_all = pd.read_csv(f'{data_path}/{c_UUID}/activities_all.csv', header=0).dropna(axis=1, how='all')
a_levels_all = pd.read_csv(f'{data_path}/{a_UUID}/levels_all.csv', header=1).dropna(axis=1, how='all')
b_levels_all = pd.read_csv(f'{data_path}/{b_UUID}/levels_all.csv', header=1).dropna(axis=1, how='all')
c_levels_all = pd.read_csv(f'{data_path}/{c_UUID}/levels_all.csv', header=1).dropna(axis=1, how='all')

# Data normalization

We want to analyze blood glucose level in time. To do so, we have two files, the first one represents differents activities (eating and sport) across the day and the second one show the blood glucose levels at differents times points. 
To join those two files, we convert their time format to UTC.

## 1 - Convert UTC+1 to UTC

In [3]:
def iso_to_utc_date(df, col):
    return [dateutil.parser.isoparse(date).astimezone(timezone.utc) for date in df[f'{col}']] 

In [4]:
a_activities_all["timestamp_start"] = iso_to_utc_date(a_activities_all, "timestamp_start")
b_activities_all["timestamp_start"] = iso_to_utc_date(b_activities_all, "timestamp_start")
c_activities_all["timestamp_start"] = iso_to_utc_date(c_activities_all, "timestamp_start")

In [5]:
a_activities_all["timestamp_start"].head()

0   2021-02-15 07:30:00+00:00
1   2021-02-15 11:45:00+00:00
2   2021-02-15 15:15:00+00:00
3   2021-02-15 16:00:00+00:00
4   2021-02-15 18:30:00+00:00
Name: timestamp_start, dtype: datetime64[ns, UTC]

## 2 - Convert dd-mm-yyyy to UTC

In [6]:
def str_to_utc(df, col):
    return [datetime.strptime(date, "%d-%m-%Y %H:%M").replace(tzinfo=timezone.utc) for date in df[f'{col}']]

In [7]:
a_levels_all["Gerätezeitstempel"] = str_to_utc(a_levels_all, "Gerätezeitstempel")
b_levels_all["Gerätezeitstempel"] = str_to_utc(b_levels_all, "Gerätezeitstempel")
c_levels_all["Gerätezeitstempel"] = str_to_utc(c_levels_all, "Gerätezeitstempel")

In [8]:
a_levels_all["Gerätezeitstempel"]

0      2021-02-18 10:57:00+00:00
1      2021-02-18 11:12:00+00:00
2      2021-02-18 11:27:00+00:00
3      2021-02-18 11:42:00+00:00
4      2021-02-18 11:57:00+00:00
                  ...           
1194   2021-02-18 09:42:00+00:00
1195   2021-02-18 09:57:00+00:00
1196   2021-02-18 10:12:00+00:00
1197   2021-02-18 10:27:00+00:00
1198   2021-02-18 10:42:00+00:00
Name: Gerätezeitstempel, Length: 1199, dtype: datetime64[ns, UTC]

In [13]:
a_levels_all = a_levels_all.sort_values(by="Gerätezeitstempel")

In [14]:
a_levels_all.drop(a_levels_all[a_levels_all.Aufzeichnungstyp > 1].index, inplace=True)

In [12]:
a_levels_all

,Gerät,Seriennummer,Gerätezeitstempel,Aufzeichnungstyp,Glukosewert-Verlauf mg/dL,Glukose-Scan mg/dL
674,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-14 16:50:00+00:00,1,NaN,53.0
851,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-14 16:52:00+00:00,0,68.0,NaN
675,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-14 16:57:00+00:00,1,NaN,63.0
676,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-14 17:03:00+00:00,1,NaN,66.0
852,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-14 17:07:00+00:00,0,72.0,NaN
...,...,...,...,...,...,...
835,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-25 16:18:00+00:00,1,NaN,98.0
671,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-25 16:19:00+00:00,0,107.0,NaN
672,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-25 16:34:00+00:00,0,100.0,NaN
673,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-25 16:49:00+00:00,0,93.0,NaN


## 3 - Combine glucose levels reading from automatic and manually scanned 

Readings of glucose levels are performed by the same sensor this is why we combined all the results

In [15]:
a_levels_all["Glucose_levels_combined"] = [y if x == 0 else z for x, y, z in zip(a_levels_all.Aufzeichnungstyp, a_levels_all["Glukosewert-Verlauf mg/dL"],a_levels_all["Glukose-Scan mg/dL"])]

# Visualization

For each individual, let's see for each meal taken the evolution of blood glucose levels from the time they have taken their meal to +3 hours

In [20]:
#Get record types as set
record_types = list(pd.concat([a_activities_all["record_type"], b_activities_all["record_type"], c_activities_all["record_type"]]).factorize()[1])
print(record_types)
re_meal = re.compile("^MEAL*")
meals_records = set(filter(re_meal.match, record_types))
print(meals_records)

['MEAL_BREAKFAST', 'MEAL_LUNCH', 'MEAL_SNACK', 'ACTVITY_EASY', 'MEAL_DINNER', 'DRINK', 'ACTVITY_MODERATE']
{'MEAL_LUNCH', 'MEAL_DINNER', 'MEAL_BREAKFAST', 'MEAL_SNACK'}


In [21]:
a_meals_records = a_activities_all.loc[a_activities_all["record_type"].isin(meals_records)]

In [22]:
a_meals_records

,id,user_id,record_type,description,timestamp_start,timestamp_end
0,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa00,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_BREAKFAST,"40 g Haferflocken, 230 g Joghurt 0,3 %, 90 g B...",2021-02-15 07:30:00+00:00,NaN
1,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa01,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_LUNCH,"98 g M�hren-Walnuss-VK-Brot, 87 g Gurke, 55 g ...",2021-02-15 11:45:00+00:00,NaN
2,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa02,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_SNACK,"Mandarine, Teel�ffel Erdnussmu�",2021-02-15 15:15:00+00:00,NaN
4,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa04,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_DINNER,"50 g BasmatiVollkorn Reis, 20g Currypaste, 15 ...",2021-02-15 18:30:00+00:00,NaN
5,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa05,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_BREAKFAST,"230 g Joghurt, 40g Haferflocken, 65 g Apfel",2021-02-17 07:15:00+00:00,NaN
7,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa07,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_LUNCH,"220 g Kohlrabi, 2 Scheiben M�hren-Walnussbrot,...",2021-02-17 11:15:00+00:00,NaN
9,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa09,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_DINNER,"Vollkornwrap mit 75 g Halloumi, 40 g Gew�rzgur...",2021-02-17 18:30:00+00:00,NaN
10,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa10,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_BREAKFAST,"100g Magerquark, 30 g Haferflocken",2021-02-19 07:30:00+00:00,NaN
11,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa11,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_LUNCH,"3 Springrolls von gestern, ohne Garnelen",2021-02-19 11:00:00+00:00,NaN
12,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaa12,aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa,MEAL_SNACK,"50g glutenfr. Nudeln, 1 EL Oliven�l, 1 Tomate,...",2021-02-19 14:45:00+00:00,NaN


In [138]:
a_glucose_levels_by_meal = []
for index, meal in a_meals_records.iterrows():
    start = meal["timestamp_start"]
    end = meal["timestamp_start"] + timedelta(hours=3)
    mask = (a_levels_all['Gerätezeitstempel'] >= start) & (a_levels_all['Gerätezeitstempel'] < end)
    df = a_levels_all.loc[mask].copy()
    df["time_delta_from_meal"] = (df["Gerätezeitstempel"] - start).apply(lambda x: x.total_seconds()/60)
    df["meal_type"] = meal["record_type"]
    a_glucose_levels_by_meal.append(df)

In [139]:
a_glucose_levels_by_meal[0]

,Gerät,Seriennummer,Gerätezeitstempel,Aufzeichnungstyp,Glukosewert-Verlauf mg/dL,Glukose-Scan mg/dL,Glucose_levels_combined,time_delta_from_meal,meal_type
902,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 07:40:00+00:00,0,81.0,NaN,81.0,10.0,MEAL_BREAKFAST
903,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 07:55:00+00:00,0,76.0,NaN,76.0,25.0,MEAL_BREAKFAST
904,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:10:00+00:00,0,74.0,NaN,74.0,40.0,MEAL_BREAKFAST
686,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:22:00+00:00,1,NaN,87.0,87.0,52.0,MEAL_BREAKFAST
905,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:25:00+00:00,0,80.0,NaN,80.0,55.0,MEAL_BREAKFAST
906,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:40:00+00:00,0,92.0,NaN,92.0,70.0,MEAL_BREAKFAST
687,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:45:00+00:00,1,NaN,108.0,108.0,75.0,MEAL_BREAKFAST
907,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 08:55:00+00:00,0,109.0,NaN,109.0,85.0,MEAL_BREAKFAST
908,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 09:10:00+00:00,0,100.0,NaN,100.0,100.0,MEAL_BREAKFAST
688,FreeStyle LibreLink,1D48A10E-DDFB-4888-8158-026F08814832,2021-02-15 09:14:00+00:00,1,NaN,85.0,85.0,104.0,MEAL_BREAKFAST


In [66]:
## historic blood glucose levels

In [67]:
px.line(a_levels_all, x='Gerätezeitstempel', y="Glucose_levels_combined").show()

In [ ]:
## historic blood glucose levels after meals

In [99]:
a_glucose_levels_by_meal

[                   Gerät                          Seriennummer  \
 902  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 903  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 904  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 686  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 905  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 906  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 687  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 907  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 908  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 688  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 909  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 910  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 911  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F08814832   
 912  FreeStyle LibreLink  1D48A10E-DDFB-4888-8158-026F0881483

In [130]:
fig = px.scatter(title="historic blood glucose levels after meals")

for i, df in enumerate(a_glucose_levels_by_meal):
    name = f"meal {i+1}"
    fig.add_scatter(x=df['time_delta_from_meal'], y=df["Glucose_levels_combined"], name=name)

fig.update_xaxes(title="time in minutes after meal")
fig.update_yaxes(title="blood glucose level in mg/dL")
fig.show()

In [ ]:
## Cluster analysis